# Imports

In [1]:
from __future__ import print_function, division
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
from IPython.display import display, HTML
%matplotlib inline
import time
import copy

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [3]:
print(torch.cuda.device_count())

4


# Data

In [4]:
FILELIST = ['chest_xray_train_val.tar.gz']

IS_ONEPANEL = True
NEED_DOWNLOAD = False
if IS_ONEPANEL:
    DATA_DIR='/onepanel/input/datasets/curae/chestx-ray8-mini/1'
else:
    import os
    DATA_DIR = 'data'
    if NEED_DOWNLOAD:
        if not os.path.exists(DATA_DIR):
            os.mkdir(DATA_DIR)
        download_files(FILELIST,DATA_DIR)

os.environ["DATADIR"] = DATA_DIR

In [5]:
# !tar xf $DATADIR/chest_xray_train_val.tar.gz -C $DATADIR

In [6]:
meta_file = os.path.join(DATA_DIR,'train_val/train_val.parquet.gz')
meta_df = pd.read_parquet(meta_file)

In [7]:
class_map = {'Cardiomegaly': 0,
             'Emphysema': 1,
             'Effusion': 2,
             'Hernia': 3,
             'Infiltration': 4,
             'Mass': 5,
             'Nodule': 6,
             'Atelectasis': 7,
             'Pneumothorax' : 8,
             'Pleural_Thickening': 9,
             'Pneumonia' : 10,
             'Fibrosis' : 11,
             'Edema': 12,
             'Consolidation': 13}
class_list = [''] * 14
for class_name, class_index in class_map.items():
  class_list[class_index] = class_name

In [8]:
def convert_to_label_string(arr):
  label = []
  for i, val in enumerate(arr):
    if val == 1:
      label.append(class_list[i])
  return '|'.join(label)

In [9]:
train_y = meta_df[meta_df['group']==0][class_map.keys()].values
test_y = meta_df[meta_df['group']==1][class_map.keys()].values

In [10]:
train_x = meta_df[meta_df['group']==0]['Image Index'].values
test_x = meta_df[meta_df['group']==1]['Image Index'].values

In [11]:
from skimage.io import imread

class ChextXrayDataset(Dataset):
    def __init__(self, folder, image_filenames, labels, transform=None):
        self.labels = torch.tensor(labels,dtype=torch.float32)
        self.folder = folder
        self.image_filenames = image_filenames
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        label = self.labels[idx]
        img = torch.tensor(
            imread(os.path.join(self.folder, self.image_filenames[idx]),
                   as_gray=True)/255.,
            dtype=torch.float32).unsqueeze(0)

#         sample = {'image': , 'label': label}

#         if self.transform:
#             sample = self.transform(sample)

        return img, label

In [56]:
train_ds = ChextXrayDataset(os.path.join(DATA_DIR,'train_val','images','train'),train_x, train_y)
test_ds = ChextXrayDataset(os.path.join(DATA_DIR,'train_val','images','val'),test_x, test_y)

In [57]:
BATCH_SIZE=512

In [58]:
dataloaders = {}
dataloaders['train'] = DataLoader(dataset=train_ds,
                         batch_size=BATCH_SIZE, shuffle=True)
dataloaders['test'] = DataLoader(dataset=test_ds,
                         batch_size=BATCH_SIZE)

In [50]:
x,y = next(iter(dataloaders['train']))
x

tensor([[[[0.0431, 0.0627, 0.0588,  ..., 0.0627, 0.0627, 0.0627],
          [0.0431, 0.0667, 0.0627,  ..., 0.0667, 0.0667, 0.0667],
          [0.0431, 0.0667, 0.0627,  ..., 0.0667, 0.0667, 0.0667],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],


        [[[0.0745, 0.0196, 0.0000,  ..., 0.1686, 0.1922, 0.2353],
          [0.0706, 0.0196, 0.0000,  ..., 0.0314, 0.0431, 0.0824],
          [0.0667, 0.0157, 0.0000,  ..., 0.0000, 0.0000, 0.0078],
          ...,
          [0.0000, 0.0039, 0.0000,  ..., 0.0000, 0.0510, 0.1333],
          [0.0000, 0.0039, 0.0000,  ..., 0.0000, 0.0510, 0.1333],
          [0.0000, 0.0039, 0.0000,  ..., 0.0000, 0.0510, 0.1333]]],


        [[[0.6745, 0.4392, 0.1765,  ..., 0.4824, 0.5529, 0.5882],
          [0.3922, 0.1451, 0.0078,  ..., 0.0706, 0.1490, 0.2471],
          [0.1804, 0.0196, 0.0000,  ..

# Model

In [51]:
import torchvision
model_ft = torchvision.models.resnet50(pretrained=False)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 14)
input_size = 224
model_ft.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)

In [52]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")
  model_ft = nn.DataParallel(model_ft)
model_ft.to(device)

Let's use 4 GPUs!


DataParallel(
  (module): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          (0): C

In [62]:
from sklearn.metrics import hamming_loss
import tqdm

def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm.tqdm_notebook(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.

                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
#                 running_corrects += torch.sum(preds == labels.data)
                running_corrects += (hamming_loss(labels.cpu(), (torch.sigmoid(outputs).data.cpu()) > 0.5)) * inputs.size(0)
                

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = 1 - running_corrects / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'test':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [63]:
params_to_update = model_ft.parameters()

optimizer_ft = optim.Adam(params_to_update, lr=0.0001)

In [ ]:
# Setup the loss fxn
# criterion = nn.BCEWithLogitsLoss()
critierion = nn.MultiLabelSoftMarginLoss()
criterion.cuda()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders, criterion, optimizer_ft, num_epochs=10)

Epoch 0/9
----------


train Loss: 0.2978 Acc: 0.8880


test Loss: 0.3035 Acc: 0.8875

Epoch 1/9
----------


train Loss: 0.2930 Acc: 0.8885


test Loss: 0.2964 Acc: 0.8872

Epoch 2/9
----------


...

test Loss: 0.2998 Acc: 0.8892

Epoch 7/9
----------


train Loss: 0.2651 Acc: 0.8959


test Loss: 0.2916 Acc: 0.8889

Epoch 8/9
----------


train Loss: 0.2582 Acc: 0.8981


test Loss: 0.3004 Acc: 0.8871

Epoch 9/9
----------


In [ ]:
%reset